<div class="alert alert-block alert-danger">
<b>warning</b> 

# RUN ALL not recommended
   </div>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.cluster import KMeans
from scipy.stats import spearmanr

pd.options.display.max_columns = None
pd.options.display.max_rows = None

import wrangle
import model

In [2]:
df = wrangle.get_explore_data()
df = wrangle.prep_data(df, features = wrangle.features)
df = wrangle.rename_data(df)
df.shape

(375734, 20)

## After importing data, I moved all featured engineered below to the top of this notebook to have them on the df before splitting into TVT

In [4]:
# defining variables
labels_n = ['low-low', 'low', 'mid', 'high']
bins = [ 0, 500, 1000, 2000, 4000]

labels_q = ['bottom_quan', 'low_quan', 'mid_quan', 'high_quan']

In [5]:
# putting engineered features onto df and test

# elevation features
df['elevation_bins']=pd.cut(df['elevation'], bins = bins, labels=labels_n, include_lowest=True)
df['elevation_quantile'] =  pd.qcut(df['elevation'], 4, labels=labels_q)
#season features
df['month']=df['startdate'].dt.month

season_groups = {
    "Autumn": [9,10,11],
    "Winter": [12,1,2],
    "Spring": [3,4,5],
    "Summer": [6,7,8],
}
df["seasons"] = (
    df["month"]
    .apply(lambda x: [k for k in season_groups.keys() if x in season_groups[k]])
    .str[0]
    .fillna("Other")
)
#regional feature
region_groups = {
    "Dry": ["BWh", "BWk", "BSh", "BSk"],
    "Temperate": ["Csa", "Csb", "Cfa", "Cfb",],
    "Continental": ["Dsb", "Dsc","Dwa","Dwb","Dfa", "Dfb", "Dfc"]
}
df["region_bins"] = (
    df["region"]
    .apply(lambda x: [k for k in region_groups.keys() if x in region_groups[k]])
    .str[0]
    .fillna("Other")
)

In [6]:
df.shape

(375734, 25)

In [7]:
train, validate, test = wrangle.split_data(df)
#df = train.copy()
train.shape, validate.shape, test.shape

((244226, 25), (75147, 25), (56361, 25))

<div class="alert alert-block alert-danger">
<b>warning</b> 

# The cells below are where I engineered features. If notbook is run-all, there may be issues with duplicates. PAY ATTENTION TO WHAT SHAPE THE DFS SHOULD BE
**RECOMMEND RUNNING ABOVE CELLS AND SKIPPING DOWN TO MARKDOWN CELL LABELED "7. MODELING"**
    </div>

## What i want to do: create data frames for each binned region and each binned elevation and each season. 
## Three features binned like
- elevation = 4
- region = 3
- season = we'll see

### steps to take 
1. pull in elevation dfs incase they are better than those created in step 2
2. use the computer to generate four bins for elevation
3. use a stat test to see which group is a better indicator for mean_temp
4. look at startdate min and max and see how many seasons those dates cover
5. create bins based on those seasons
6. take in Rae's bins for region
7. model off of the bins in various combinations
8. model off of a single narrow bin and all other contest features

# 1. elevation bins

In [ ]:
# these are the four naomi-made dataframes
elevation_low_low=train[train.elevation < 500.0]
elevation_low=train[(train.elevation > 500.0)&(train.elevation <1000)]
elevation_mid = train[(train.elevation >1000.0) & (train.elevation < 2000.0)]
elevation_high = train[train.elevation > 2000.0]

In [ ]:
# this is the train dataframe with a new column that labels the elevation based on bin labels

labels = ['low-low', 'low', 'mid', 'high']
bins = [ 0, 500, 1000, 2000, 4000]

train['elevation_bins'] = pd.cut(train['elevation'] , bins=bins, labels=labels, include_lowest=True)


In [ ]:
validate.shape

In [ ]:
# wraping this code in a function takes forever and breaks the host

validate['elevation_bins']=pd.cut(validate['elevation'], bins = bins, labels=labels, include_lowest=True)


In [ ]:
validate.shape

In [ ]:
test['elevation_bins']=pd.cut(test['elevation'], bins = bins, labels=labels, include_lowest=True)


In [ ]:
test.shape

# 2. program generated bins

In [ ]:
labels = ['bottom_quan', 'low_quan', 'mid_quan', 'high_quan']
train['elevation_quantile'] =  pd.qcut(train['elevation'], 4, labels=labels)
train.shape

In [ ]:
validate['elevation_quantile'] =  pd.qcut(validate['elevation'], 4, labels=labels)
validate.shape


In [ ]:
test['elevation_quantile'] = pd.qcut(test['elevation'], 4, labels=labels)
test.shape

# 3. stat test for significance

In [ ]:
# ask mark for help

# 4. how many seasons?


In [ ]:
train.startdate.max()

In [ ]:
train.startdate.min()

Counting September as an Autumn Month the break down is as such:
- 2014 Sep-Nov = 1
- 2014 Dec-2015 Feb = 2
- 2015 Mar-May = 3
- 2015 Jun-Aug = 4
- 2015 Sep-Nov = 5
- 2015 Dec-2016 Feb = 6
- 2016 Mar-May = 7
- 2016 Jun-Aug = 8
*** eight seasons, two full years, two of each season




# 5. creating bins based on seasons
### do i want eight bins, one for each season? or four bins each with two seasons?


In [ ]:
train.head()

In [3]:
train['month']=train['startdate'].dt.month

NameError: name 'train' is not defined

In [ ]:
# binning for season. Each bin holds two years of seasons
season_groups = {
    "Autumn": [9,10,11],
    "Winter": [12,1,2],
    "Spring": [3,4,5],
    "Summer": [6,7,8],
}
train["seasons"] = (
    train["month"]
    .apply(lambda x: [k for k in groups.keys() if x in groups[k]])
    .str[0]
    .fillna("Other")
)

In [ ]:
df.head()

In [ ]:
# creating four dataframes based on season
autumn_train = train[train.seasons == 'Autumn']
winter_train = train[train.seasons == 'Winter']
spring_train = train[train.seasons == 'Spring']
summer_train = train[train.seasons == 'Summer']

In [ ]:
autumn_train.shape, winter_train.shape, spring_train.shape, summer_train.shape

In [ ]:
def season_bins(df):
    df['month']=df['startdate'].dt.month
    
    season_groups = {
        "Autumn": [9,10,11],
        "Winter": [12,1,2],
        "Spring": [3,4,5],
        "Summer": [6,7,8],
    }
    df["seasons"] = (
        df["month"]
        .apply(lambda x: [k for k in season_groups.keys() if x in season_groups[k]])
        .str[0]
        .fillna("Other")
    )
    return df
    

In [ ]:
season_bins(df)

In [ ]:
df.head()

# 6. Take in Region bins

In [ ]:
region_groups = {
    "Dry": ["BWh", "BWk", "BSh", "BSk"],
    "Temperate": ["Csa", "Csb", "Cfa", "Cfb",],
    "Continental": ["Dsb", "Dsc","Dwa","Dwb","Dfa", "Dfb", "Dfc"]
}
train["region_bins"] = (
    train["region"]
    .apply(lambda x: [k for k in region_groups.keys() if x in region_groups[k]])
    .str[0]
    .fillna("Other")
)

In [ ]:
train.region_bins.value_counts()

In [ ]:
df.shape

In [ ]:
# defining variables
labels_n = ['low-low', 'low', 'mid', 'high']
bins = [ 0, 500, 1000, 2000, 4000]

labels_q = ['bottom_quan', 'low_quan', 'mid_quan', 'high_quan']



In [ ]:
# putting engineered features onto df
# elevation features
df['elevation_bins']=pd.cut(df['elevation'], bins = bins, labels=labels_n, include_lowest=True)
df['elevation_quantile'] =  pd.qcut(df['elevation'], 4, labels=labels_q)
#season features
df['month']=df['startdate'].dt.month

season_groups = {
    "Autumn": [9,10,11],
    "Winter": [12,1,2],
    "Spring": [3,4,5],
    "Summer": [6,7,8],
}
df["seasons"] = (
    df["month"]
    .apply(lambda x: [k for k in season_groups.keys() if x in season_groups[k]])
    .str[0]
    .fillna("Other")
)
#regional feature
region_groups = {
    "Dry": ["BWh", "BWk", "BSh", "BSk"],
    "Temperate": ["Csa", "Csb", "Cfa", "Cfb",],
    "Continental": ["Dsb", "Dsc","Dwa","Dwb","Dfa", "Dfb", "Dfc"]
}
df["region_bins"] = (
    df["region"]
    .apply(lambda x: [k for k in region_groups.keys() if x in region_groups[k]])
    .str[0]
    .fillna("Other")
)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
train.shape

# 7. modeling

In [ ]:
drivers = list(df.columns)

drivers.remove('startdate')
drivers.remove('mean_temp')
drivers.remove('month')
# model is improved with month scaled as a feature
drivers.remove('region_bins')

In [ ]:
X_train, y_train, X_validate, y_validate, X_test, y_test = model.prep_for_model(df, validate, test, 'mean_temp', drivers)


In [ ]:
X_train.shape, y_train.shape, X_validate.shape, y_validate.shape, X_test.shape, y_test.shape

In [ ]:
X_train.head()

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
model.regression_models(X_train, y_train, X_validate, y_validate)

# 8. Narrow bin modeling
- run through OLS correlation thing for each dataframe and see if different features correlate differently between them

In [ ]:
# **be sure to use binning before running season and region as the data is not encoded in the original**

#elevation dataframes
elevation_low_low=df[df.elevation < 500.0]
elevation_low=df[(df.elevation > 500.0)&(df.elevation <1000)]
elevation_mid = df[(df.elevation >1000.0) & (df.elevation < 2000.0)]
elevation_high = df[df.elevation > 2000.0]

# season dataframes
autumn_df = df[df.seasons == 'Autumn']
winter_df = df[df.seasons == 'Winter']
spring_df = df[df.seasons == 'Spring']
summer_df = df[df.seasons == 'Summer']

# region dataframes
dry_df = df[df.region_bins == "Dry"]
temperate_df = df[df.region_bins == "Temperate"]
continental_df = df[df.region_bins == 'Continental']

In [ ]:
dry_df.shape

In [ ]:
df.region_bins.value_counts()